##**Домашнее задание по теме "Функции потерь и оптимизация"**

**Задание:**

**Цель:** изучить применение методов оптимизации для решения задачи классификации

**Описание задания:**

В домашнем задании необходимо применить полученные знания в теории оптимизации и машинном обучении для реализации логистической регрессии.

**Этапы работы:**

1. Загрузите данные. Используйте датасет с ирисами. Его можно загрузить непосредственно из библиотеки Sklearn. В данных оставьте только 2 класса: Iris Versicolor, Iris Virginica.
2. Самостоятельно реализуйте логистическую регрессию, без использования метода LogisticRegression из библиотеки. Можете использовать библиотеки pandas, numpy, math для реализации. Оформите в виде функции. Оформите в виде класса с методами.
3. Реализуйте метод градиентного спуска. Обучите логистическую регрессию этим методом. Выберете и посчитайте метрику качества. Метрика должна быть одинакова для всех пунктов домашнего задания. Для упрощения сравнения выберете только одну метрику.
4. Повторите п. 3 для метода скользящего среднего (Root Mean Square Propagation, RMSProp).
5. Повторите п. 3 для ускоренного по Нестерову метода адаптивной оценки моментов (Nesterov–accelerated Adaptive Moment Estimation, Nadam).
6. Сравните значение метрик для реализованных методов оптимизации. Можно оформить в виде таблицы вида |метод|метрика|время работы| (время работы опционально). Напишите вывод.

**Решение:**

In [ ]:
# Импортируем необходимые библиотеки
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris

In [ ]:
# Загружаем датафрейм
df = pd.DataFrame(load_iris(as_frame=True).data)
df['target'] = load_iris().target
df = df.loc[df.target != 0]
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
50,7.0,3.2,4.7,1.4,1
51,6.4,3.2,4.5,1.5,1
52,6.9,3.1,4.9,1.5,1
53,5.5,2.3,4.0,1.3,1
54,6.5,2.8,4.6,1.5,1


In [ ]:
# Выбираем необходимые признаки и переформатируем метки классов
X, y = df[['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']], df['target']
y -= 1
X.shape, y

((100, 4),
 50     0
 51     0
 52     0
 53     0
 54     0
       ..
 145    1
 146    1
 147    1
 148    1
 149    1
 Name: target, Length: 100, dtype: int64)

In [ ]:
# Напишем класс Логистической регрессии
class logistic_regression:
    def __init__(self, iterations=250, learning_rate=0.01, eps = 0.007, epsilon=1e-30):
        self.iterations = iterations
        self.learning_rate = learning_rate
        self.eps = eps
        self.epsilon = epsilon

    # Предобработка данных
    def data_prep(self, X, y):
        from sklearn.preprocessing import StandardScaler

        scaler = StandardScaler()
        self.X = scaler.fit_transform(X)
        # столбец из единиц для работы со свободными весами
        self.X = np.c_[np.ones(len(self.X)), self.X]
        self.y = y


    # Функция сигмоиды
    def sigmoid(self, x):
        return 1/(1+np.exp(-x))

    # Проверка точности
    def loop_exit(self, cur_W):
        if np.linalg.norm(cur_W - self.next_W) <= self.eps:
              return True

    # Функция потерь
    def logloss(self):
        l_loss0 = np.sum(np.log(1 - self.y_proba[self.y == 0]) + self.epsilon)
        l_loss1 = np.sum(np.log(self.y_proba[self.y == 1]) + self.epsilon)
        return -(l_loss0 + l_loss1)/len(self.y)

    # Расчет градиента
    def grad(self):
        grad = self.X.T @ (self.y_proba - self.y)
        return grad

    # Установка стартовых весов
    def start_weights(self):
        np.random.seed(42)
        self.weights = np.random.randn(self.X.shape[1])
        return self.weights

    # Градиентный спуск
    def gradient_descent(self):
        self.next_W = self.start_weights()
        for i in range(self.iterations):
            cur_W = self.next_W

            self.y_proba = self.sigmoid(self.X @ self.next_W)
            # смена направления на противоположное
            self.next_W = cur_W - self.learning_rate * self.grad()

            if self.loop_exit(cur_W):
                break

        y_class = np.where(self.y_proba >= 0.5, 1, 0)
        accuracy = (y_class == y).sum() / len(y)
        print(f"Всего {i+1} итераций")
        print(f"Logloss {self.logloss()}")
        print(f"Accuracy {accuracy}")

    # RMSProp
    def RMSProp(self, decay_rate=0.9):
        self.next_W = self.start_weights()
        grad_squared = np.zeros_like(self.next_W)

        for i in range(self.iterations):
            cur_W = self.next_W

            self.y_proba = self.sigmoid(self.X @ self.next_W)
            # расчет квадрата градиента
            grad_squared = decay_rate * grad_squared + (1 - decay_rate) * np.square(self.grad())
            # вычисление новых весов
            self.next_W = cur_W - self.learning_rate * self.grad() / (np.sqrt(grad_squared) + self.epsilon)

            if self.loop_exit(cur_W):
                break

        y_class = np.where(self.y_proba >= 0.5, 1, 0)
        accuracy = (y_class == self.y).sum() / len(self.y)
        print(f"Всего {i+1} итераций")
        print(f"Accuracy {accuracy}")

    # Nadam
    def nadam(self, beta_1=0.9, beta_2=0.999):

        m_t = np.zeros(self.weights.shape)
        v_t = np.zeros(self.weights.shape)
        self.next_W = self.start_weights()

        for i in range(self.iterations):
            cur_W = self.next_W

            self.y_proba = self.sigmoid(self.X @ self.next_W)
            # обновляем первый и второй моменты
            m_t = beta_1*m_t + (1-beta_1)*self.grad()
            v_t = beta_2*v_t + (1-beta_2)*self.grad()**2
            # исправляем смещение
            m_t_hat = m_t / (1 - beta_1**(i+1))
            v_t_hat = v_t / (1 - beta_2**(i+1))
            # вычисляем коррекционный коэффициент
            n_correction = self.learning_rate / (np.sqrt(v_t_hat) + self.epsilon)
            # вычисляем следующую точку
            self.next_W = cur_W - n_correction * (beta_1*m_t_hat + (1-beta_1)*self.grad())

            if self.loop_exit(cur_W):
                break

        y_class = np.where(self.y_proba >= 0.5, 1, 0)
        accuracy = (y_class == self.y).sum() / len(self.y)
        print(f"Всего {i+1} итераций")
        print(f"Accuracy {accuracy}")

In [ ]:
model = logistic_regression()

In [ ]:
model.data_prep(X, y)

In [ ]:
%time model.gradient_descent()

Всего 230 итераций
Logloss 0.07128996304462287
Accuracy 0.97
CPU times: user 35.1 ms, sys: 0 ns, total: 35.1 ms
Wall time: 36.4 ms


In [ ]:
%time model.RMSProp()

Всего 250 итераций
Accuracy 0.99
CPU times: user 93.3 ms, sys: 3.38 ms, total: 96.6 ms
Wall time: 91.9 ms


In [ ]:
%time model.nadam()

Всего 218 итераций
Accuracy 0.99
CPU times: user 149 ms, sys: 12.9 ms, total: 162 ms
Wall time: 154 ms


По точности предсказаний ближе всего RMSProp() и Nadam() - 99%, немного ниже показатели точности у Градиентного спуска - 97%. По затраченному времени лидирует Градиентный спуск - 36,4 мс., затем идет RMSProp() - 91,9 мс.и Nadam - 154 мс. Интересно, что Градиентный спуск в несколько раз быстрее, чем RMSProp(). При этом нет корреляции затраченного времени и количества итераций. Например, у Градиентного спуска итераций больше, чем у Nadam (230 против 218), однако время ниже (36,4 мс. против 154 мс.). Выбор метода будет обусловлен условиями конкретной задачи и тем, какие ресурсы необходимо сэкономить при ее реализации.